# Setup

In [37]:
num = '106'
windows = False

In [38]:
import pandas as pd
import json
import os

# Defining Columns of Interest

In [39]:
BASIC_COLUMNS = {'bill_id', 'bill_type',  'congress', 'introduced_at', 'number', 'status', 'status_at', 'subjects_top_term', 'updated_at'}

LIST_COLUMNS  = {'committees', 'subjects'}

DICT_COLUMNS  = {'sponsor' : ['thomas_id', 'bioguide_id'], 
                 'history' : ['awaiting_signature', 'enacted', 'vetoed', 'active', 'house_passage_result', 'senate_passage_result']}

IGNORE = {'actions', 'amendments', 'cosponsors', 'enacted_as', 'official_title', 'popular_title', 'related_bills', 'short_title', 'summary', 'titles', 'updated_at'}

df_cols = list(BASIC_COLUMNS) + sum(DICT_COLUMNS.values(), []) + list(LIST_COLUMNS)

def createRow(json_dict):
    row_dict = {}
    for c in BASIC_COLUMNS:
        if c in json_dict: row_dict[c] = json_dict[c]
    for l in LIST_COLUMNS:
        if l == 'committees': # SPECIAL CASE
            comms = []
            for cs in json_dict[l]: comms.append(cs['committee'])
            row_dict[l] = comms.__repr__()
        elif l in json_dict: row_dict[l] = json_dict[l].__repr__()
    for di,fields in DICT_COLUMNS.items():
        if di in json_dict:
            for field in fields:
                if json_dict[di] == None: continue
                if field in json_dict[di]:
                    row_dict[field] = json_dict[di][field]
    return row_dict

In [40]:
BASIC_COLUMNS1 = {'bill_id', 'bill_type',  'congress', 'introduced_at', 'number', 'status', 'status_at', 'subjects_top_term'}

LIST_COLUMNS1  = {'committees', 'subjects'}

DICT_COLUMNS1  = {'sponsor' : ['thomas_id', 'bioguide_id'], 
                 'history' : ['awaiting_signature', 'enacted', 'vetoed', 'active', 'house_passage_result', 'senate_passage_result']}

IGNORE1 = {'actions', 'amendments', 'cosponsors', 'enacted_as', 'official_title', 'popular_title', 'related_bills', 'short_title', 'summary', 'titles', 'updated_at'}

df_cols1 = list(BASIC_COLUMNS1) + sum(DICT_COLUMNS1.values(), []) + list(LIST_COLUMNS1)

def createRow1(json_dict):
    row_dict = {}
    for c in BASIC_COLUMNS1:
        if c in json_dict: row_dict[c] = json_dict[c]
    for l in LIST_COLUMNS1:
        if l == 'committees': # SPECIAL CASE
            comms = []
            for cs in json_dict[l]: comms.append(cs['committee'])
            row_dict[l] = comms.__repr__()
        elif l in json_dict: row_dict[l] = json_dict[l].__repr__()
    for di,fields in DICT_COLUMNS1.items():
        if di in json_dict:
            if json_dict[di] == None: continue
            for field in fields:
                if field in json_dict[di]:
                    row_dict[field] = json_dict[di][field]
    return row_dict

In [41]:
IGNORE1 = {'actions', 'amendments', 'cosponsors', 'enacted_as', 'official_title', 'popular_title', 'related_bills', 'short_title', 'summary', 'titles', 'updated_at'}

df_cols2 = {'bill_id', 'introduced_at', 'congress', 'thomas_id', 'bioguide_id', 'primary_sponsor',  'name', 'state'}
solo_cols = {'thomas_id', 'bioguide_id', 'name', 'state'}
df_cols2l = ['bill_id', 'introduced_at', 'congress', 'thomas_id', 'bioguide_id', 'primary_sponsor',  'name', 'state']


def createSponsorDF(json_dict):
    df = pd.DataFrame(columns=df_cols2l,index=['bill_id'])
    
    # sponsor
    row_dict = {'bill_type':json_dict['bill_type'], 'bill_id':json_dict['bill_id'], 'introduced_at':json_dict['introduced_at'], 'congress':json_dict['congress'], 'primary_sponsor':True}
    for c in solo_cols:
        if json_dict['sponsor'] == None: continue
        if c in json_dict['sponsor']:
                row_dict[c] = json_dict['sponsor'][c]
    rdf = pd.DataFrame(row_dict,index=[1])
    df = pd.concat([df, rdf],ignore_index=True)
    
    
    for csp in json_dict['cosponsors']:
        row_dict = {'bill_type':json_dict['bill_type'], 'bill_id':json_dict['bill_id'], 'introduced_at':json_dict['introduced_at'], 'congress':json_dict['congress'], 'primary_sponsor':False}
        for c in solo_cols:
             if c in csp:
                    row_dict[c] = csp[c]
        rdf = pd.DataFrame(row_dict,index=[1])
        df = pd.concat([df, rdf],ignore_index=True)
    
        
    return df.tail(-1)

In [42]:
count = 0
if windows: rootdir = '.\\'+num+'\\bills\\'
else: rootdir = './bills/'
df1 = pd.DataFrame(columns=df_cols1,index=['bill_id'])
df2 = pd.DataFrame(columns=df_cols2l,index=['bill_id'])

i = 1
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file[-5:] != '.json': continue  
        count += 1
print(count)

10840


In [43]:
if windows: rootdir = '.\\'+num+'\\bills\\'
else: rootdir = './bills/'
df1 = pd.DataFrame(columns=df_cols1,index=['bill_id'])
df2 = pd.DataFrame(columns=df_cols2l,index=['bill_id'])

i = 1
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file[-5:] != '.json': continue              
        f = open(os.path.join(subdir, file))
        d = json.load(f)
        f.close()
        
        tdf1 =  pd.DataFrame(createRow1(d),index=[i])
        df1 = pd.concat([df1, tdf1],ignore_index=True)
        
        df2 = pd.concat([df2, createSponsorDF(d)],ignore_index=True)
        
        i += 1 
        if i%1000 == 0: print(i)

df1 = df1.set_index('bill_id').tail(-1)
df2 = df2.set_index('bill_id').tail(-1)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [44]:
df1.to_excel(num+"-Bills.xlsx")  
df2.to_excel(num+'-Sponsors.xlsx')